In [14]:
from beakerx import *
import sqlalchemy as sq
print(sq.__version__)

1.2.17


In [15]:
import pandas as pd
prices=pd.read_csv("data/price.csv", index_col=0, header=0, parse_dates=True)

In [16]:
from pyutil.sql.base import Base
from pyutil.sql.interfaces.symbols.symbol import Symbol, SymbolType
from pyutil.sql.session import session 

with session('sqlite:///file.db', echo=False, base=Base) as s:
    for key, data in prices.items():
        symbol = Symbol(name=key, group=SymbolType.equities, internal=key)
        # upsert the prices
        symbol.upsert_price(data.dropna())
        s.add(symbol)
        
    # this will raise an error as a Symbol with the same name has already been defined
    # same issue if we execute this cell a second time...
    # you may need to delete file.db by hand
    # symbol = Symbol(name="A", group=SymbolType.equities, internal="Peter Maffay")
    # session.add(symbol)
    
    # commit is called whenever we leave the scope, so no need to do this explicitly here
    # s.commit()


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: productinterface.discriminator, productinterface.name [SQL: 'INSERT INTO productinterface (name, discriminator) VALUES (?, ?)'] [parameters: ('A', 'symbol')] (Background on this error at: http://sqlalche.me/e/gkpj)

In [17]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    for symbol in s.query(Symbol):
        print(symbol)
        print(symbol.price.tail(2))

symbol(A)
2015-04-21    1202.34
2015-04-22    1200.59
Name: A, dtype: float64
symbol(B)
2015-04-21    27850.49
2015-04-22    27964.84
Name: B, dtype: float64
symbol(C)
2015-04-17    53954.79
2015-04-20    53761.27
Name: C, dtype: float64
symbol(D)
2015-04-21    2651.41
2015-04-22    2638.95
Name: D, dtype: float64
symbol(E)
2015-04-20    2100.40
2015-04-21    2097.29
Name: E, dtype: float64
symbol(F)
2015-04-21    3719.38
2015-04-22    3742.77
Name: F, dtype: float64
symbol(G)
2015-04-20    3059.73
2015-04-21    3065.53
Name: G, dtype: float64


In [23]:
with session('sqlite:///file.db', echo=False, base=Base) as s:
    a = s.query(Symbol).filter(Symbol.name=="A").one()
    b = s.query(Symbol).filter(Symbol.name=="B").one()
    symbols = [a,b]
    t = Symbol.frame(symbols)
    print(t)
    h = Symbol.history(symbols).tail(5)
    print(h)
    



          Name    Sector
Symbol                  
symbol(A)    A  Equities
symbol(B)    B  Equities
            symbol(A)  symbol(B)
2015-04-16    1198.56   27739.71
2015-04-17    1204.27   27653.12
2015-04-20    1195.88   27094.93
2015-04-21    1202.34   27850.49
2015-04-22    1200.59   27964.84
